<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, <a href="https://www.hse.ru/en/staff/sara/" target="_blank">Saraa Ali</a>  ©2025 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🧬Genomics</font>](https://www.kaggle.com/competitions/25-hse-genomics/rules)**. [**Instructions**](https://colab.research.google.com/drive/1owkYjuRGkx050LQnM3b3yTzd0Dr2XbeV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
# from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 25-hse-genomics # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

In [ ]:
!nvidia-smi --query-gpu=gpu_name,memory.total,memory.free,memory.used --format=csv # GPU specs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
%%time
%%capture
%reset -f
!pip -q install -U sentence-transformers > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer as SBERT
from sklearn.svm import SVC, LinearSVC, NuSVC
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*5): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
pd.set_option('display.max_rows', 100, 'display.max_columns', 100, 'display.max_colwidth', 100, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 2.4 s, sys: 515 ms, total: 2.91 s
Wall time: 4.41 s


In [2]:
vX = pd.read_csv('testX/testX.csv').set_index('id')
tYX = pd.read_csv('trainYX/trainYX.csv').set_index('id')
vX

,DNA
id,
100000,TTGATTAATAAGATTCCTTGACACCCTTTGTAAAGTTTCTATTTCGTGTGAAATATCTATCTCTTCAAATCCTTTTAATTTATCTAGGTATTTGCT...
100001,ATTAGTAACGGAGGATTTACTAGATGTTTGGATTTATATTCTAATTTTATTCAGGTGGAAGGGATTGTTTTATGATTCAATAGTATACAGAGAATA...
...,...
119998,CGTCGGCATGCTCGGGCAGTGCGGCGGGCCAGCAGCGTGCCAGTTGTCGCGGGGCGGCCGGGCATCGCGGCGCCGGGCGGCAGCACTCCCGCGAAG...
119999,GCGAGGGCACGAAGGCACGACGGCAACGGCGGCGAGGAGCGCTGTGGCAACCGTCTCCGCGTTTGCGTGCGTACAGCCGAGAGCTGGTTCGCGCAG...


In [ ]:
!pip install biopython -q

In [3]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 300 sec. Good luck!


❗Do not modify the setup above.

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

Students: Keep all your definitions, code, documentation **between** ⏳ symbols.

## Task 1. Preprocessing Pipeline

**Pipeline elements**
1. **6-mer character n-grams** via `CountVectorizer`  
   Capture local sequence motifs typical for bacterial species.

2. **Compositional DNA features**  
   GC content, AT/GC ratio, melting temperature, sequence length.

3. **Physicochemical protein features**  
   DNA → protein (frame 1) → `ProteinAnalysis` (instability, aromaticity, isoelectric point, GRAVY, helix/turn/sheet fractions, protein length).

4. **Z-curve features**  
   Global sequence-shape descriptors: x/y/z axes, magnitude, group ratios (purine/pyrimidine, amino/keto).

5. **PCA (100 components)**  
   Reduce dimensionality, remove noise, speed up SVM.

6. **Concatenation of all features**  
   `train_all` / `test_all` formed by combining k-mers + handcrafted blocks.

---

### Why these elements?
- k-mers model local patterns; handcrafted features capture biological properties not visible to n-grams.  
- PCA stabilizes training and improves generalization.

### Effectiveness evaluation
- Stratified 80/20 split, `accuracy_score` on validation.  
- Compared pipelines: k-mers only vs. k-mers + engineered features, with/without PCA.

### What didn’t work?
- **Positional one-hot encoding** → extremely large sparse matrices, slow training, no accuracy gain.  
- Dense + PCA pipeline performed best.



## Task 2. Modeling Approach

**Final model**
```python
LinearSVC(C=0.5, class_weight='balanced', dual=False,
          random_state=42, max_iter=3000)
```
### Why this model?

- Linear SVM is well-suited after rich feature engineering + PCA.

- C=0.5 reduces overfitting.

- class_weight='balanced' handles class imbalance.

- dual=False is efficient when features < samples after PCA.

### Effectiveness evaluation

- Validation accuracy on held-out split.

- Checked convergence, stability across different C and feature subsets.

### What else was tried?

- SVC and sparse hstack versions were tested but slower and not better.

- Final choice optimized accuracy + runtime.


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer, OneHotEncoder
from typing import List, Tuple
from sklearn.decomposition import PCA
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Seq import Seq
from sklearn.metrics import accuracy_score

In [4]:
def kmer_char_ngrams(sequences: pd.Series, k: int, 
                     normalize: bool = True) -> pd.DataFrame:

    vectorizer = CountVectorizer(
        analyzer='char',      # Character-level analysis
        ngram_range=(k, k),   # Extract k-grams only
        lowercase=False       # Case-sensitive
    )
    
    kmer_matrix = vectorizer.fit_transform(sequences)
    
    if normalize:
        normalizer = Normalizer(norm='l1')
        kmer_matrix = normalizer.transform(kmer_matrix)
    
    feature_names = [f"{k}mer_{name}" for name in vectorizer.get_feature_names_out()]
    df = pd.DataFrame(
        kmer_matrix.toarray(),
        index=sequences.index,
        columns=feature_names
    )
    
    return df

In [30]:
train = kmer_char_ngrams(tYX['DNA'], k=6, normalize=True).reset_index()
test = kmer_char_ngrams(vX['DNA'], k=6, normalize=True).reset_index()


In [31]:
print("train shape: ", train.shape)
print("test shape: ", test.shape)

train shape:  (100000, 4097)
test shape:  (20000, 4097)


In [7]:
train_seqs = tYX['DNA'].astype(str).tolist()
test_seqs = vX['DNA'].astype(str).tolist()
all_seqs = train_seqs + test_seqs

def pad_sequence(seq, length):
    # Right-pad with 'N' (standard placeholder for unknown nucleotide)
    return list(seq.ljust(length, 'N'))

max_len = max(len(s) for s in all_seqs)
min_len = min(len(s) for s in all_seqs)

train_chars = [pad_sequence(s, max_len) for s in train_seqs]
test_chars = [pad_sequence(s, max_len) for s in test_seqs]
all_chars = train_chars + test_chars

In [8]:
ohe = OneHotEncoder(sparse_output=True, handle_unknown='ignore', dtype=np.int8)

print("   Fitting OneHotEncoder on padded sequences...")
ohe.fit(all_chars)

train_pos = ohe.transform(train_chars)
test_pos = ohe.transform(test_chars)

   Fitting OneHotEncoder on padded sequences...


,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.int8'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [10]:
print(f"train_pos shape: {train_pos.shape}")
print(f"test_pos shape: {test_pos.shape}")

train_pos shape: (100000, 2002)
test_pos shape: (20000, 2002)


In [14]:
def compute_compositional_features(sequences):
    features = []
        
    for seq in sequences:
        seq_upper = seq.upper()
        length = len(seq_upper)
        
        # Count nucleotides
        g_count = seq_upper.count('G')
        c_count = seq_upper.count('C')
        a_count = seq_upper.count('A')
        t_count = seq_upper.count('T')
        
        # GC content (percentage)
        gc_content = (g_count + c_count) / length if length > 0 else 0
        
        # AT/GC ratio
        gc_total = g_count + c_count
        at_total = a_count + t_count
        at_gc_ratio = at_total / gc_total if gc_total > 0 else 0
        
        # Melting temperature (Wallace rule for short sequences, nearest-neighbor for accuracy)
        # Using simplified Wallace rule: Tm = 2(A+T) + 4(G+C)
        melting_temp = 2 * (a_count + t_count) + 4 * (g_count + c_count)
        
        features.append({
            'gc_content': gc_content,
            'at_gc_ratio': at_gc_ratio,
            'melting_temp': melting_temp,
            'gc_count': g_count + c_count,
            'at_count': a_count + t_count,
            'seq_length': length
        })

    return pd.DataFrame(features).reset_index()

# Compute compositional features
train_comp = compute_compositional_features(tYX['DNA'])
test_comp = compute_compositional_features(vX['DNA'])



In [15]:
print(f"train comp shape: {train_comp.shape}")
print(f"test comp shape: {test_comp.shape}")

train comp shape: (100000, 7)
test comp shape: (20000, 7)


In [17]:
def compute_physicochemical_features(sequences: pd.Series) -> pd.DataFrame:

    features = []
    
    for seq in sequences:
        try:
            # Translate DNA to protein (take first reading frame)
            dna_seq = Seq(seq.upper())
            protein_seq = str(dna_seq.translate(to_stop=False))
            
            protein_seq = protein_seq.replace('*', '')
            
            if len(protein_seq) > 0:
                protein_analysis = ProteinAnalysis(protein_seq)
                
                mol_weight = protein_analysis.molecular_weight()
                
                aromaticity = protein_analysis.aromaticity()
                
                instability = protein_analysis.instability_index()
                
                isoelectric_point = protein_analysis.isoelectric_point()
                
                gravy = protein_analysis.gravy()
                
                helix, turn, sheet = protein_analysis.secondary_structure_fraction()
                
                features.append({
                    'mol_weight': mol_weight,
                    'aromaticity': aromaticity,
                    'instability_index': instability,
                    'isoelectric_point': isoelectric_point,
                    'gravy_hydrophobicity': gravy,
                    'helix_fraction': helix,
                    'turn_fraction': turn,
                    'sheet_fraction': sheet,
                    'protein_length': len(protein_seq)
                })
            else:
                features.append({
                    'mol_weight': 0,
                    'aromaticity': 0,
                    'instability_index': 0,
                    'isoelectric_point': 0,
                    'gravy_hydrophobicity': 0,
                    'helix_fraction': 0,
                    'turn_fraction': 0,
                    'sheet_fraction': 0,
                    'protein_length': 0
                })
        except Exception as e:
            features.append({
                'mol_weight': 0,
                'aromaticity': 0,
                'instability_index': 0,
                'isoelectric_point': 0,
                'gravy_hydrophobicity': 0,
                'helix_fraction': 0,
                'turn_fraction': 0,
                'sheet_fraction': 0,
                'protein_length': 0
            })
    
    return pd.DataFrame(features).reset_index()

train_phys = compute_physicochemical_features(tYX['DNA'])
test_phys = compute_physicochemical_features(vX['DNA'])


/Users/pavel-sardak/Documents/Учеба/Tracker/Tasks/STUDY-38/.venv/lib/python3.14/site-packages/Bio/Seq.py:2877: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
/Users/pavel-sardak/Documents/Учеба/Tracker/Tasks/STUDY-38/.venv/lib/python3.14/site-packages/Bio/Seq.py:2877: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [18]:
print(f"train_phys shape: {train_phys.shape}")
print(f"test_phys shape: {test_phys.shape}")

train_phys shape: (100000, 10)
test_phys shape: (20000, 10)


In [20]:
def compute_zcurve_features(sequences: pd.Series) -> pd.DataFrame:

    features = []
    
    for seq in sequences:
        seq_upper = seq.upper()
        length = len(seq_upper)
        
        # Count nucleotides
        a_count = seq_upper.count('A')
        c_count = seq_upper.count('C')
        g_count = seq_upper.count('G')
        t_count = seq_upper.count('T')
        
        if length > 0:
            # Z-curve coordinates (normalized by sequence length)
            x = (a_count + g_count - c_count - t_count) / length  # Purine vs Pyrimidine
            y = (a_count + c_count - g_count - t_count) / length  # Amino vs Keto
            z = (a_count + t_count - g_count - c_count) / length  # Weak vs Strong H-bonds
            
            # Additional Z-curve derived features
            # Phase angle in xy-plane
            phase_xy = np.arctan2(y, x) if x != 0 else 0
            
            # Phase angle in xz-plane
            phase_xz = np.arctan2(z, x) if x != 0 else 0
            
            # Phase angle in yz-plane
            phase_yz = np.arctan2(z, y) if y != 0 else 0
            
            # Magnitude in 3D space
            magnitude = np.sqrt(x**2 + y**2 + z**2)
            
            # Cumulative Z-curve features (mean of cumulative sums)
            cumsum_x = 0
            cumsum_y = 0
            cumsum_z = 0
            
            for i, nuc in enumerate(seq_upper):
                if nuc in 'AG':
                    cumsum_x += 1
                elif nuc in 'CT':
                    cumsum_x -= 1
                    
                if nuc in 'AC':
                    cumsum_y += 1
                elif nuc in 'GT':
                    cumsum_y -= 1
                    
                if nuc in 'AT':
                    cumsum_z += 1
                elif nuc in 'GC':
                    cumsum_z -= 1
            
            # Normalize cumulative sums
            cumsum_x_norm = cumsum_x / length
            cumsum_y_norm = cumsum_y / length
            cumsum_z_norm = cumsum_z / length
            
            features.append({
                'zcurve_x': x,
                'zcurve_y': y,
                'zcurve_z': z,
                'zcurve_phase_xy': phase_xy,
                'zcurve_phase_xz': phase_xz,
                'zcurve_phase_yz': phase_yz,
                'zcurve_magnitude': magnitude,
                'zcurve_cumsum_x': cumsum_x_norm,
                'zcurve_cumsum_y': cumsum_y_norm,
                'zcurve_cumsum_z': cumsum_z_norm,
                'purine_ratio': (a_count + g_count) / length,
                'pyrimidine_ratio': (c_count + t_count) / length,
                'amino_ratio': (a_count + c_count) / length,
                'keto_ratio': (g_count + t_count) / length
            })
        else:
            # Handle empty sequences
            features.append({
                'zcurve_x': 0, 'zcurve_y': 0, 'zcurve_z': 0,
                'zcurve_phase_xy': 0, 'zcurve_phase_xz': 0, 'zcurve_phase_yz': 0,
                'zcurve_magnitude': 0,
                'zcurve_cumsum_x': 0, 'zcurve_cumsum_y': 0, 'zcurve_cumsum_z': 0,
                'purine_ratio': 0, 'pyrimidine_ratio': 0,
                'amino_ratio': 0, 'keto_ratio': 0
            })
    
    return pd.DataFrame(features).reset_index()

train_zcurve = compute_zcurve_features(tYX['DNA'])
test_zcurve = compute_zcurve_features(vX['DNA'])


In [21]:
print(f"train zcurve shape: {train_zcurve.shape}")
print(f"test zcurve shape: {test_zcurve.shape}")

train zcurve shape: (100000, 15)
test zcurve shape: (20000, 15)


In [18]:
from scipy.sparse import hstack

In [32]:
# Now concatenate
test_all = pd.concat([test, test_zcurve, test_comp, test_phys], axis=1)
train_all = pd.concat([train, train_zcurve, train_comp, train_phys], axis=1)

# train_all = hstack([train, train_zcurve, train_comp, train_phys])
# test_all = hstack([test, test_zcurve, test_comp, test_phys])

print(f"\nFinal train shape: {train_all.shape}")
print(f"Final test shape: {test_all.shape}")



Final train shape: (100000, 4129)
Final test shape: (20000, 4129)


In [ ]:
# apply PCA
pca = PCA(n_components=100)
train_all = pca.fit_transform(train_all)
test_all = pca.transform(test_all)

In [26]:
X_tr, X_val, y_tr, y_val = train_test_split(
    train_all, tYX['Y'], test_size=0.2, random_state=42, stratify=tYX['Y']
)

print("Training LinearSVC (C=0.5)...")
# C=0.5 is a sweet spot for hybrid features (prevents overfitting on the dense positional data)
clf = LinearSVC(
    C=0.5,
    class_weight='balanced',
    dual=False,
    random_state=42,
    max_iter=3000
)
# X_tr.columns = X_tr.columns.astype(str)
# X_val.columns = X_val.columns.astype(str)
clf.fit(X_tr, y_tr)

# Validate
val_preds = clf.predict(X_val)
acc = accuracy_score(y_val, val_preds)
print(f"✓ Validation Accuracy: {acc:.5f}")

Training LinearSVC (C=0.5)...


,penalty,'l2'
,loss,'squared_hinge'
,dual,False
,tol,0.0001
,C,0.5
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,verbose,0
,random_state,42


✓ Validation Accuracy: 0.97610


In [34]:
print("\nRetraining on FULL dataset...")
clf_final = LinearSVC(
    C=0.5,
    class_weight='balanced',
    dual=False,
    random_state=42,
    max_iter=3000
)
clf_final.fit(train_all, tYX['Y'])

print("Generating predictions...")
test_preds = clf_final.predict(test_all)

submission_df = pd.DataFrame({'id': vX.index, 'y': test_preds})
submission_df.to_csv('submission.csv', index=False)
print(f"✓ Submission saved to submission.csv ({len(submission_df)} rows)")
print(submission_df.head())


Retraining on FULL dataset...


,penalty,'l2'
,loss,'squared_hinge'
,dual,False
,tol,0.0001
,C,0.5
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,verbose,0
,random_state,42


Generating predictions...
✓ Submission saved to submission.csv (20000 rows)
        id  y
0   100000  0
1   100001  0
..     ... ..
3   100003  1
4   100004  1

[5 rows x 2 columns]


# **References:**

1. Stack Overflow. (2025). Stack Overflow: Where developers learn, share, & build careers. https://stackoverflow.com
2. Yandex Education. (n.d.). Machine Learning Handbook. Retrieved October 5, 2025, from https://education.yandex.ru/handbook/ml\n,
3. NumPy Developers. (2025). NumPy documentation. https://numpy.org/doc\n

<font color=green><h4><b>$\epsilon$. LLM Documentation if used</b></h4></font>

We have used ChatGPT (GPT-5) as a technical assistant during implementation. Specifically, it helped to:

1. Draft and refine preprocessing patterns  
   (e.g., structuring k-mer extraction, designing compositional / physicochemical feature blocks, and ensuring stable PCA integration).

2. Suggest alternative feature representations  
   (e.g., positional one-hot encoding, sparse vs. dense concatenation, handling empty or error-prone sequences).

3. Provide debugging guidance  
   (e.g., resolving issues with sparse matrices, dimensionality mismatches, and PCA transformations).

4. Verify syntax compatibility between Colab, scikit-learn, BioPython, and Kaggle submission formatting.

The LLM provided syntax-level guidance, but the selection of features, modeling decisions, PCA configuration, and the final classifier choice were determined independently by the authors.


<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

# 💡**Starter Ideas**

1. Learn about DNA [&#127910;](https://www.youtube.com/results?search_query=nucleotides+genes+amino+acids+)
1. Try a larger training sample.
1. Try longer training DNA strings, but SBERT may have a cap on string length, so you might split DNA into several strings and then concatenate or average resulting vectors
1. Try other pretrained SBERT models. Note that DNA sequence uses ACGT letters, but many other models were trained on multilingual text. So, you might prefer those that were trained on mostly ASCII.
1. SBERT is trained on word tokens (typically, separated by spaces), but DNA sequence has no spaces. Try placing spaces after every character or some semantically meaningful subsequences (this might require more domain knowledge).
1. Try Google's [USE](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3) embedding models
1. Try Facebook's [LASER](https://github.com/facebookresearch/LASER) and [others](https://tfhub.dev/s?module-type=text-language-model).
1. Try [Enformer](https://tfhub.dev/deepmind/enformer/1) for gene expressions. See [DeepMind paper](https://deepmind.com/blog/article/enformer).
1. Try building your own embeddings on the given sequences. SBERT and other packages make it easy (just a few lines), but it may take too much time.
1. Assess distribution of character patterns (single, doubles, triplets, ...). For example, an ACGT string generates AC, CG, GT doubles and ACG and CGT triplets. Does one class have more subsequences of some type? This might be a feature in your model.
1. Try features built as counts of subsequences (singles, doubles, triplets, ...). Consider EDA first.
1. Concatenate or otherwise combine multiple embeddings derived from each gene string
1. Learn from [*The genetic code*](https://www.khanacademy.org/science/ap-biology/gene-expression-and-regulation/translation/a/the-genetic-code-discovery-and-properties), Khan Academy.
1. Learn from [*Apply Machine Learning Algorithms for Genomics Data Classification*](https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723)
1. Learn from [*Efficient counting of k-mers in DNA sequences using a bloom filter*](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-333) Páll Melsted et al. 2011
1. Try [Byte Pair Encoding](https://www.derczynski.com/papers/archive/BPE_Gage.pdf) and [SentencePiece](https://arxiv.org/pdf/1808.06226.pdf) to auto identification of "important" [k-mers](https://en.wikipedia.org/wiki/K-mer) (substrings)